In [33]:
import requests
import collections
from operator import itemgetter
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime

In [32]:
font = '| font=Courier | size=14'
site = 'https://www.oddsshark.com'
lgs  = collections.defaultdict(dict)
lgs['UFC']['odds'] = requests.get(
    'https://io.oddsshark.com/ticker/ufc', 
    headers = {
        'referer': site
    }
)

for league, active in lgs['UFC']['odds'].json()['leagues'].items():
    lgs[league.upper()]['futures'] = BeautifulSoup(requests.get(f"{site}/{league}/odds/futures").content, 'html.parser')
    if active==True:
        lgs[league.upper()]['odds'] = requests.get(
            'https://io.oddsshark.com/ticker/' + league, 
            headers = {
                'referer': site
            }
        ).json()['matchups']

In [16]:
def simple_odds(odds):
    for game in odds:
        if game['type'] == 'date':
            print('--', game['date']['fullday'], game['date']['month'], game['date']['day'])
        if game["type"] == "matchup":
            if game['status'].startswith('FINAL'):
                print('----', game['status'], '| font=Courier | href=' + site + game['matchup_link'])
                print('----', game['away_short_name'], game['away_score'], '| font=Courier')
                print('----', game['home_short_name'], game['home_score'], '| font=Courier')
            else:
                print('----', game['status'], '| font=Courier | href=' + site + game['matchup_link'])
                print('----', game['away_short_name'], game['away_odds'] if game['away_odds'].startswith('-') else '+' + game['away_odds'], '| font=Courier')
                print('----', game['home_short_name'], game['home_odds'] if game['home_odds'].startswith('-') else '+' + game['home_odds'], '| font=Courier')

In [23]:
for sport in lgs:
    if sport != 'UFC':
        print(sport)
        if 'odds' in lgs[sport]:
            simple_odds(lgs[sport]['odds'])
        print('---')

NHL
-- Tuesday Apr 20
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/boston-buffalo-odds-april-20-2021-1402266
---- BOS 2 | font=Courier
---- BUF 0 | font=Courier
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/columbus-florida-odds-april-20-2021-1367346
---- CLB 1 | font=Courier
---- FLA 5 | font=Courier
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/new-york-new-york-odds-april-20-2021-1367356
---- NYR 1 | font=Courier
---- NYI 6 | font=Courier
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/carolina-tampa-bay-odds-april-20-2021-1367361
---- CAR 4 | font=Courier
---- TB 1 | font=Courier
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/new-jersey-pittsburgh-odds-april-20-2021-1380386
---- NJ 6 | font=Courier
---- PIT 7 | font=Courier
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/detroit-dallas-odds-april-20-2021-1369916
---- DET 2 | font=Courier
---- DAL 5 | font=Courier
---- FINAL | font=Cour

In [ ]:
lg_tks['UFC']

In [ ]:
print('UFC')
for fight in lg_tks['UFC']:
    if fight['type'] == 'event':
        print('--', fight['event'])
    if fight["type"] == 'matchup':
        if not fight['status']:
            print('----', fight['event_date'][11:])
            print('----', fight['away_name'], fight['away_odds'] if fight['away_odds'].startswith('-') else '+' + fight['away_odds'], '| font=Courier')
            print('----', fight['home_name'], fight['home_odds'] if fight['home_odds'].startswith('-') else '+' + fight['home_odds'], '| font=Courier')
        else:
            print('----', 'FINAL')
            print('----', fight['away_name'], fight['status'] if fight['away_name'] == fight['winner'] else '', '| font=Courier')
            print('----', fight['home_name'], fight['status'] if fight['home_name'] == fight['winner'] else '', '| font=Courier')

In [ ]:
print(soup.prettify())

In [15]:
def process_odds_section(soup, html_type, html_class):
    data = soup.find_all(html_type, class_=html_class)
    init_list = []
    daysofweek = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    for elem in data:
        if not elem.get_text().startswith(tuple(daysofweek)):
            init_list.append(elem.get_text().strip())
    return init_list

In [88]:
def create_futures_data(teams, opening, currents):
    
    futures_trios = []
    for a, b, c in zip(*[iter(currents)]*3):
        futures_trios.append([a,b,c])
    
    rows = []
    odds_inc = 0
    sports = ['MLB','NFL','NHL','NBA','College Basketball','College Football']
    for team in teams:
        if not team.startswith(tuple(sports)):
            rows.append(Future(future, team,
                         opening[odds_inc],
                         futures_trios[odds_inc][0],
                         futures_trios[odds_inc][1],
                         futures_trios[odds_inc][2]))
            odds_inc += 1
        else:
            rows.append([team])
            future = team
    return rows

In [13]:
def print_future_odds(sport):
    print('--', sport)
    opening_odds = process_odds_section(lg_futs[sport], 'div', ['op-item op-future-item', 'op-item op-future-item '])
    current_odds = process_odds_section(lg_futs[sport], 'div', 'op-item op-future-item no-vegas')
    team_list = process_odds_section(lg_futs[sport], ['div','span'], ['align-bottom',
                                                                      'op-team baseball op-odd',
                                                                      'op-team baseball op-even',
                                                                      'op-team basketball op-odd',
                                                                      'op-team basketball op-even',
                                                                      'op-team football op-odd',
                                                                      'op-team football op-even',
                                                                      'op-team hockey op-odd',
                                                                      'op-team hockey op-even'])
    all_odds = create_futures_data(team_list, opening_odds, current_odds)
    for odds in all_odds:
        if len(odds) > 1:
            print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format(odds[0], odds[1], odds[2], odds[3], odds[4], font))
        else:
            print('----', odds[0], font)
            print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format('Team','Opening','Bovada','BetOnline.AG','Sports Betting',font))

In [42]:
def print_all_odds(lgs):
    for sport in lgs:
        if sport != 'UFC':
            print(sport)
            if 'odds' in lgs[sport]:
                simple_odds(lgs[sport]['odds'])
        opening_odds = process_odds_section(lgs[sport]['futures'], 'div', ['op-item op-future-item', 'op-item op-future-item '])
        current_odds = process_odds_section(lgs[sport]['futures'], 'div', 'op-item op-future-item no-vegas')
        team_list = process_odds_section(lgs[sport]['futures'], ['div','span'], ['align-bottom',
                                                                          'op-team baseball op-odd',
                                                                          'op-team baseball op-even',
                                                                          'op-team basketball op-odd',
                                                                          'op-team basketball op-even',
                                                                          'op-team football op-odd',
                                                                          'op-team football op-even',
                                                                          'op-team hockey op-odd',
                                                                          'op-team hockey op-even'])
        all_odds = create_futures_data(team_list, opening_odds, current_odds)
        for odds in all_odds:
            if len(odds) > 1:
                print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format(odds[0], odds[1], odds[2], odds[3], odds[4], font))
            else:
                print('----', odds[0], font)
                print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format('Team','Opening','Bovada','BetOnline.AG','Sports Betting',font))

In [89]:
opening_odds = process_odds_section(lgs['NHL']['futures'], 'div', ['op-item op-future-item', 'op-item op-future-item '])
current_odds = process_odds_section(lgs['NHL']['futures'], 'div', 'op-item op-future-item no-vegas')
team_list = process_odds_section(lgs['NHL']['futures'], ['div','span'], ['align-bottom',
                                                                          'op-team baseball op-odd',
                                                                          'op-team baseball op-even',
                                                                          'op-team basketball op-odd',
                                                                          'op-team basketball op-even',
                                                                          'op-team football op-odd',
                                                                          'op-team football op-even',
                                                                          'op-team hockey op-odd',
                                                                          'op-team hockey op-even'])
all_odds = create_futures_data(team_list, opening_odds, current_odds)

In [83]:
class Future:
    def __init__(self, future, team, opening, bovada, betonline, sportsbetting):
        self.future = future
        self.team = team
        self.opening = opening
        self.bovada = bovada
        self.betonline = betonline
        self.sportsbetting = sportsbetting
    def __repr__(self):
        return repr((self.future, self.team, self.opening, self.bovada, self.betonline, self.sportsbetting))

In [51]:
for odds in all_odds_sorted:
    if len(odds) > 1:
        print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format(odds[0], odds[1], odds[2], odds[3], odds[4], font))
    else:
        print('----', odds[0], font)
        print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format('Team','Opening','Bovada','BetOnline.AG','Sports Betting',font))

------ Anaheim                 +8000 +150000        +75000         +75000| font=Courier | size=14
------ Anaheim                 +1400 +150000       +100000        +100000| font=Courier | size=14
------ Arizona                 +4000   +6000         +7500          +7500| font=Courier | size=14
------ Arizona                 +1200  +25000        +10000         +10000| font=Courier | size=14
------ Boston                  +1200   +1400         +1400          +1400| font=Courier | size=14
------ Boston                   +250    +250          +260           +260| font=Courier | size=14
------ Buffalo                 +8000 +150000       +300000        +300000| font=Courier | size=14
------ Buffalo                 +1200 +100000       +250000        +250000| font=Courier | size=14
------ Calgary                 +3000  +10000         +5000          +5000| font=Courier | size=14
------ Calgary                  +546  +50000         +4000          +4000| font=Courier | size=14
------ Carolina     

In [97]:
int(all_odds[10].bovada)

4000